In this notebook, you'll implement object detection using the very power YOLO (You Only Look Once) model.   

Many of the ideas in this notebook are described in the two YOLO papers: [Redmon et al., 2016](https://arxiv.org/abs/1506.02640) and [Redmon and Farhadi, 2016](https://arxiv.org/abs/1612.08242). 

**By the end of this, you will be able to:**
-  Detect objects in a car detection dataset
-  Implement non-max suppression to increase accuracy
-  Implement intersection over union (IoU) to evaluate bounding box predictions
-  Handle bounding boxes, a type of image annotation popular in deep learning.

### Package 


In [3]:
import os 

In [ ]:
branch_path = 'Chapter04-Convolutional-Neural-Networks/Object_Detection/W3A1'
os.chdir(os.path.join(os.getcwd(), branch_path))
os.getcwd()

In [ ]:
import argparse

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image

import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor

from tensorflow.keras.models import load_model

from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

%matplotlib inline

### 1 - Problem Statement

You are working on a self-driving car. Go you! As a critical component of this project, you'd like to first build a car detection system. To collect data, you've mounted a camera to the hood (meaning the front) of the car, which takes pictures of the road ahead every few seconds as you drive around. 

<center>
<video width="400" height="200" src="nb_images/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Pictures taken from a car-mounted camera while driving around Silicon Valley. <br> Dataset provided by <a href="https://www.drive.ai/">drive.ai</a>.
</center></caption>

You've gathered all these images into a folder and labelled them by drawing bounding boxes around every car you found. Here is an example of what your bounding boxes look like:

<center>
<img src="nb_images/box_label.png" style="width:500px;height:250;">
<caption><center> <u><b>Figure 1</u></b>: Definition of a box<br> </center></caption>
</center>

If there are 80 classes you want the object detector to recgonize, you can represent the class label c either as an integer from 1 to 80, or as an 80-dimensional vector (with 80 numbers) one component of which is 1 and the rest are all 0. The video lectures used the latter representation;  in this notebook, you'll use both representations, depending on which is more convenient for a particular step.  

Because the YOLO model is very computationally expensive to train, the pre-trained weights are already loaded for you to use. 



## 2 - YOLO

"You Only Look Once" (YOLO) is a popular algorithm because it achieves high accuracy while also being able to run in real-time. 